In [2]:
# load libraries
import numpy as np
import pandas as pd
import sys, pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split, GridSearchCV

Remove the irrelevant features or those that have repeated information such as age in age groups when age is available as a continuous variable. Recode missing values, refused to answers to '123456789', and if the answer was none as an asnwer to, for example, 'how many drinks did you have in the last 30 days?' to 0.

In [3]:
brfss = pd.read_csv("brfss2014.csv", index_col = 0,encoding='latin1')
brfss = brfss.drop(['fmonth','imonth','iday','idate','seqno','x.psu','colghous','ladult','flshtmy2','dlyother',
                    'alcday5','rcsbirth','x.ststr','x.strwt','x.rawrake','x.wt2rake','x.dualcor','x.llcpwt2',
                    'x.llcpwt','x.ageg5yr','x.age65yr','x.age.g','htm4','x.bmi5','x.rfbmi5','x.drnkdy4','x.smoker3',
                   'x.rfdrhv4','x.rfdrmn4','x.rfdrwm4','x.rfseat2','x.mam502y','x.mam5021','x.rfblds3','x.col10yr',
                   'x.cllcpwt','x.casthm1','x.ltasth1','x.mrace1'],axis=1)
brfss = brfss.fillna(123456789)

brfss = brfss.replace([77,777,7777],123456789)
brfss =  brfss.replace([99,999,9999], 123456789)
brfss = brfss.replace([88,888,8888], 0)

print(brfss.shape)
sys.getsizeof(brfss)/1024**2
# keep only the recalculated or imputed variables

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(464664, 240)


854.3703231811523

Choose AIDS testing status (x.aidtst3) as label, its values are : 1 tested, 2 not tested, [7,9,123456789] refused, don't know and missing, respectively.

In [4]:
label_var = 'x.aidtst3'
brfss = brfss.loc[~brfss[label_var].isin([9,7,123456789])]
label = brfss[label_var]
print(brfss.shape)
sys.getsizeof(brfss)/1024**2

(418626, 240)


769.7209854125977

In [5]:
label.value_counts()

2.0    301619
1.0    117007
Name: x.aidtst3, dtype: int64

Further select ratio scale and categorical variables for the analysis

In [6]:
ratio_scale_variable_labels = ['sleptim1','x.age80','x.impnph','htin4','wtkg3','drocdy3.','x.drnkmo4']
ratio_scale_variables = brfss[ratio_scale_variable_labels]
categorical_variable_labels = ['x.imprace','x.impeduc','x.impmrtl','x.impcsex','x.asthms1','x.incomg',
                                'x.rfseat3','x.flshot6','x.pneumo2','x.bmi5cat','x.rfmam2y','x.denvst2','x.rfsmok3']
categorical_variables = brfss[categorical_variable_labels].astype('int64')
#print(categorical_variables.shape)
variables = pd.concat([categorical_variables, ratio_scale_variables], axis = 1)
print(variables.shape)
sys.getsizeof(variables)/1024**2

(418626, 20)


67.0711441040039

The first 13 features are categorical, the remaining 7 are on interval/ratio scale. Create one hot encoder for the categoricals.And split into 0.67 training set, 0.33 test sets

In [7]:
ohe = OneHotEncoder(categorical_features = range(13))
ohe_variables = ohe.fit_transform(variables)
# get back feature name
uniq_vals = categorical_variables.apply(lambda x: x.value_counts()).unstack()
uniq_vals = uniq_vals[~uniq_vals.isnull()]
categorical_ohe_feature_labels = list(uniq_vals.index.map('{0[0]}_{0[1]}'.format))
feature_names = np.hstack([categorical_ohe_feature_labels, ratio_scale_variable_labels])

X_train, X_test, y_train, y_test = train_test_split(ohe_variables, label, test_size=0.33, random_state=1234)

# Run: Fit a depth 5 decision tree

In [9]:
clf = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=61, max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=25, min_samples_split=100,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

clf.fit(X_train, y_train)
export_graphviz(clf, out_file = 'tree.dot', feature_names = feature_names, class_names = ['tested','not-tested'],
               rounded =  True, filled = True)
#!dot -Tpng tree.dot -o tree.png


pickle.dump(clf, open('decision_tree_AIDS_5lvl.pickle','wb'))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


# Not Run yet: Find optimal tree parameters by cross-validation

In [10]:
clf = DecisionTreeClassifier(random_state=123)
params = {'max_depth':[3,4,5,10,12,15], 'min_samples_split':[2,5,10,50,100], 'min_samples_leaf':[1,5,10,25],
          'max_features':[5,10,20,30,61], 'class_weight':[None,'balanced']}
gd = GridSearchCV(clf, params, cv = 10, verbose = 1, n_jobs = 4)
gd.fit(X_train, y_train)

Fitting 10 folds for each of 1200 candidates, totalling 12000 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    6.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   28.4s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  5.4min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  7.8min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed: 11.7min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed: 25.5min
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed: 46.3min
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed: 79.4min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed: 83.1min
[Parallel(n_jobs=4)]: Done 8442 tasks      | elapsed: 87.9min
[Parallel(n_jobs=4)]: Done 9792 tasks      | elapsed: 99.0min
[Parallel(n_jobs=4)]: Done 11242 tasks      | elapsed: 128.2min
[Parallel(n_jobs=4)]: Done 12000 out of 12000 | elapsed: 157.2min finish

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'max_depth': [3, 4, 5, 10, 12, 15], 'min_samples_split': [2, 5, 10, 50, 100], 'min_samples_leaf': [1, 5, 10, 25], 'max_features': [5, 10, 20, 30, 61], 'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [11]:
gd.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=61, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=25, min_samples_split=100,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [12]:
gd.best_score_

0.7464908246250165

In [16]:
clf_best = gd.best_estimator_
clf_best.fit(X_train, y_train)

export_graphviz(clf_best, out_file = 'opt_tree.dot', feature_names = feature_names, class_names = ['tested','not-tested'],
               rounded =  True, filled = True)
#!dot -Tpng opt_tree.dot -o opt_tree.png


pickle.dump(clf_best, open('decision_tree_AIDS_opt.pickle','wb'))